_Version Log: using openai to understand compressed sensing in python_

In [ ]:
import numpy as np
from scipy.fftpack import fft, idct
from l1magic import l1eq_pd

def dct_magic(k, m, n, b, Fs):
    A = np.zeros((m, n))
    for j in range(m):
        ek = np.zeros(n)
        ek[k[j]] = 1
        A[j, :] = idct(ek)

    y = np.linalg.pinv(A).dot(b)

    x = l1eq_pd(y, A, A.T, b, 5e-3, 32)

    xl2 = y

    l1 = x
    l1 = abs(l1) / max(abs(l1))

    xl1 = idct(x)

    kd = Fs / 2 * np.linspace(0, 1, n)

    return l1, xl1, kd

def ft_magic(k, m, n, b, Fs):
    PHI = np.fft.fft(np.eye(n))
    S = np.zeros((m, n))

    for j in range(m):
        ek = np.zeros(n)
        ek[k[j]] = 1
        S[j, :] = ek

    A = S * (1/n) * PHI.T

    y = np.linalg.pinv(A).dot(b)

    x12 = (1/n) * np.real(PHI.T.dot(y))

    x = l1eq_pd(y, A, A.T, b, 5e-3, 32)

    l1 = 2 * x[:n//2]

    xl1 = (1/n) * np.real(PHI.T.dot(x))

    l1 = abs(l1) / max(abs(l1))

    kd = Fs / 2 * np.linspace(0, 1, n // 2)

    return l1, xl1, kd

def single_sided_ft(x, Fs, sin_theta_incident):
    N = len(x)
    X = fft(x)
    f = Fs / 2 * np.linspace(0, 1, N // 2)
    f = f[:, np.newaxis]
    X = 2 * np.abs(X[:N // 2])
    X = X / max(X)
    f = 1.0 / (f / (2 * sin_theta_incident))
    return X, f

def single_sided_ft_b(x, Fs, sin_theta_incident):
    N = len(x)
    X = fft(x)
    f = Fs / 2 * np.linspace(0, 1, N // 2)
    f = f[:, np.newaxis]
    X = 2 * np.abs(X[:N // 2])
    X = X / max(X)
    f = 1.0 / (f / (2 * sin_theta_incident))
    return X, f

# Now you can call the functions with your data and parameters.

In [ ]:
import numpy as np
from scipy.fftpack import fft, idct
import random

# Read in data
with open('150_points_500um_pp_0.25THz_width_287times2.txt', 'r') as fileID:
    data = np.loadtxt(fileID)
f = (data[:, 1] * 0.001).reshape(-1, 1)
t = (data[:, 0] * 0.001).reshape(-1, 1)
n = len(f)
spacing = 0.25  # cm spacing

# Parameters to change
det_size = 120  # Detector size in points
pnum = det_size * 1  # Choose the number of sparse points to sample
position = [0.6]  # Change

# Do we want DCT (mode=0) or FFT (mode=1)?
mode = 1

# Experiment conditions
dx = 500  # Width of pixels in micrometers
Fs = 1.0 / dx
theta = 4  # Angle between beams
sin_theta_incident = np.sin(np.deg2rad(theta))
det_size_cm = det_size * dx * 1e-4  # Convert to cm
pnum = det_size_cm / spacing  # Every 0.5 cm

# Noisy observations
sigma = 0.00001
e = sigma * np.random.randn(n, 1)
f = f + e

# If normal
p = len(position)
m = int(np.ceil(pnum)) * p
m_one_det = int(np.ceil(pnum))
b_matrix = np.zeros((p, m_one_det))
k_matrix = np.zeros((p, m_one_det))

# The FFT of the input
spectrum_fft, um_fft = single_sided_ft(f, Fs, sin_theta_incident)

for i in range(p):
    k_p = int(np.ceil(n * position[i]))  # Find the position of the detector
    k_p_min = k_p - int(np.ceil(det_size / 2))  # Position of detector
    rand = random.sample(range(det_size), m_one_det)
    k = np.array([x + k_p_min for x in rand])  # Not random

    if any(k > n):
        print('Detector is too big')
        break
    elif any(k < 0):
        print('Detector is too small')
        break

    k_matrix[i, :] = k
    b = f[k]
    b_matrix[i, :] = b

# A = rows of DCT matrix with indices of the chosen points
k_mat = k_matrix[0, :]
b_mat = b_matrix[0, :]
k = k_mat
b = b_mat
m, n = b.shape

A = np.zeros((m, n))

if mode == 0:
    print('DCT magic routine')
    l1, xl1, kd = dct_magic(k, m, n, b, Fs)
else:
    print('FFT magic routine')
    l1, xl1, kd = ft_magic(k, m, n, b, Fs)

b_fft_m, um_b_fft = single_sided_ft_b(b, Fs, sin_theta_incident)

um = 1.0 / (kd / (2 * sin_theta_incident))

# Plot
import matplotlib.pyplot as plt

t = t * (1.0 / 0.001)
plt.figure(1)
plt.subplot(3, 1, 1)
plt.plot(t, f, 'b', markersize=5)
plt.plot(t[k], b, 'or', markersize=12)
plt.title('f = signal, b = random sample')
plt.xlabel('length along detector cm')

plt.subplot(3, 1, 2)
GHz_fft = (1.0 / um) * 299.792458 * 1e3
plt.plot(GHz_fft, spectrum_fft, 'b-', GHz_fft, l1, 'r-')
plt.xlabel('GHz')
plt.axis([0, 500, 0, 1])

if mode == 0:
    plt.title('x = l1 solution, A*x = b using DCT basis')
else:
    plt.title('x = l1 solution, A*x = b using FFT basis')

plt.subplot(3, 1, 3)
plt.plot(t, xl1, 'r-', t, f, 'b-')
plt.xlabel('length along detector cm')

if mode == 0:
    plt.title('DCT(x)')
else:
    plt.title('Reconstructed time domain signal from x = l1 solution')

plt.show()

# Write out
B = np.vstack((GHz_fft, l1))
with open('experimental_peaks.txt', 'w') as fileID:
    np.savetxt(fileID, B.T, fmt='%6.9f %12.9f')
